# SS02 Day1-8 All Data
2017-12-11 

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import statsmodels.api as sm 
from asp.load import Trial

/Applications/anaconda2/envs/pyqt4_for_ploting/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [1]:
%matplotlib qt

In [3]:
totalTrial = 16
t = []
for trialId in range(1,totalTrial+1):
    i = trialId-1
    t.append(Trial(2, trialId))
    t[i].readDecoder()
    t[i].readConductor()
    t[i].gaitSegmentation()
    t[i].RValueByGaitAllJoint()
    t[i].rTrainTestbyGait()

/Applications/anaconda2/envs/pyqt4_for_ploting/lib/python2.7/site-packages/numpy/lib/function_base.py:3167: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Applications/anaconda2/envs/pyqt4_for_ploting/lib/python2.7/site-packages/numpy/lib/function_base.py:3168: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


### smooth the r values. This is not used anymore 

In [1]:
# signal = []
# for i in range(len(t)):
#     signal.append(sm.nonparametric.lowess(t[i].rLeftHipByGait[:,1], t[i].rLeftHipByGait[:,0], frac=0.01))

## A figure of all r values in each gait in each trial
Each dot is the r-value in one gait cycle

In [4]:
joint = 4 # 0: time. 1. right hip. 2: right knee. 3: right ankle. 4: left hip. 5: left knee. 6: left ankle

In [7]:
fig, axarr = plt.subplots(len(t)/2,2, figsize=(7,9))
# fig.dpi = 200
# fig.set_size_inches(600./fig.dpi,800./fig.dpi)

for i in range(len(t)):
    # remove gaits that were before 30s
    firstStep = 0
    while t[i].rLeftHipByGait[firstStep,0]<30:
        firstStep += 1
    
    axarr[i/2,i%2].plot(t[i].rAllJointByGait[firstStep:,0], t[i].rAllJointByGait[firstStep:,joint],'.',markersize=2)
    axarr[i/2,i%2].set_xlim([0, 1250])
    axarr[i/2,i%2].set_ylim([-1,1])
    # a vertical line to mark the beginning of EEG control
    bciOnset = 630
    axarr[i/2,i%2].plot([bciOnset, bciOnset], axarr[i/2,i%2].get_ylim(), 'k-', lw=2)
#     axarr[i/2,i%2].autoscale(enable=True, axis='y', tight=True)    
    
    # remove the frame
    axarr[i/2,i%2].spines['top'].set_visible(False)
    axarr[i/2,i%2].spines['right'].set_visible(False)
    
    axarr[i/2,i%2].grid(b=True, which='major', axis='y')
    axarr[i/2,i%2].set_ylabel('Trial '+str(i+1), fontsize=9)
    if i<len(t)-2: # unless it's the last two figures (on the last row), remove xticks.
        axarr[i/2,i%2].set_xticks([])
    else:
        axarr[i/2,i%2].set_xlabel('Time (sec)')
    if i%2==1:
        axarr[i/2,i%2].set_yticklabels([])
    
fig.subplots_adjust(hspace=0.3, wspace=0.24, top=0.95, bottom=0.08, right=0.95)    
fig.text(0.01, 0.5, 'Performance (r-value)', va='center', rotation='vertical')

fig.show()
# fig.savefig('Figures/r_SS02Day1-8.png', dpi=600, format='png', transparent=True)
# fig.savefig('Figures/r_SS02Day1-8.eps', dpi=600, format='eps', transparent=True)

In [46]:
plt.close('all')

## Compare the median r value changes over sessions

Day5 Trial2: stopped walking at 860s to fix the lose ankle goniometer connector

In [10]:
#     joint = 3 #This variable is also given in the previous plot.

rTrainAll = []
rTestAll = []
color1, color2 = '#2C7BB6', '#D7191C'    # colors are from http://colorbrewer2.org/

for i in range(0,len(t)):
    rTrainAll.append(t[i].rTrainMedian[joint-1]) # -1 because rTrainMean doesn't have the time column
    rTestAll.append(t[i].rTestMedian[joint-1])
x = range(1, len(rTrainAll)+1)

fig = plt.figure(figsize=(7,5))
plt.plot(x, rTrainAll, marker='o',color=color1, linestyle='none', label='Train')
plt.plot(x, rTestAll, marker='o', color=color2, markerfacecolor='none', linestyle='none', label='Test')

#     if i==0: # for plotting legend
#         plt.plot(i+1, rTrainMedian, marker='o',color='darkorange', label='Train', linestyle='none')
#         plt.plot(i+1, rTestMedian, marker='o', color='darkorange', markerfacecolor='none',label='Test', linestyle='none')
#     else:
#         plt.plot(i+1, rTrainMedian, marker='o',color='darkorange', linestyle='none')
#         plt.plot(i+1, rTestMedian, marker='o', color='darkorange', markerfacecolor='none', linestyle='none')

        
slopeTrain, interceptTrain, r_value, p_value, std_err = scipy.stats.linregress(x, rTrainAll)
# # # Trial 10 has very bad value because stopped in middle
slopeTest, interceptTest, r_value, p_value, std_err = scipy.stats.linregress(x[0:9]+x[10:-1], rTestAll[0:9]+rTestAll[10:-1])

# fit line: train
x1=1
x2=len(t)
y1=slopeTrain*x1+interceptTrain
y2=slopeTrain*x2+interceptTrain
plt.plot([x1,x2],[y1,y2],linewidth=3, color=color1)        
# fit line: test
x1=1
x2=len(t)
y1=slopeTest*x1+interceptTest
y2=slopeTest*x2+interceptTest
plt.plot([x1,x2],[y1,y2],linewidth=3, color=color2)         
        
plt.grid('on')
plt.xlabel('Trial')
plt.ylabel('Performance (r-value)')
plt.legend()
plt.show()

# fig.savefig('Figures/SS02Day1-8_r_vs_trial.png', dpi=600, format='png', transparent=True)
# fig.savefig('Figures/SS02Day1-8_r_vs_trial.eps', dpi=600, format='eps', transparent=True)

## Boxplot of r value changes over sessions


In [11]:
#     joint = 3 #This variable is also given in the previous plot.
data_a = []
data_b = []
for i in range(len(t)):
    # train
    data_a.append(t[i].rTrain[:,joint])
    # test
    data_b.append(t[i].rTest[:,joint])

In [12]:
# data_a = [[1,2,5], [5,7,2,2,5], [7,2,5]]
# data_b = [[6,4,2], [1], [2,3,5,1]]

temp = range(len(t))
ticks = map(lambda x: str(x+1), temp)

def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)

plt.figure(figsize=(7,4))

bpl = plt.boxplot(data_a, positions=np.array(xrange(len(data_a)))*2.0-0.3, sym='', widths=0.3, showbox=True, showcaps=False)
bpr = plt.boxplot(data_b, positions=np.array(xrange(len(data_b)))*2.0+0.3, sym='', widths=0.3, showcaps=False)
set_box_color(bpl, color1) 
set_box_color(bpr, color2)

# draw temporary red and blue lines and use them to create a legend
plt.plot([], c=color1, label='Train')
plt.plot([], c=color2, label='Test')
plt.legend()

plt.xticks(xrange(0, len(ticks) * 2, 2), ticks)
plt.xlim(-2, len(ticks)*2)
plt.ylim(-1, 1)
# plt.tight_layout()
plt.xlabel('Trial')
plt.ylabel('Performance (r-value)')

plt.subplots_adjust(right=0.96)

In [13]:
fig.savefig('Figures/SS02Day1-8_r_boxplot.png', dpi=600, format='png', transparent=True)
fig.savefig('Figures/SS02Day1-8_r_boxplot.eps', dpi=600, format='eps', transparent=True)

In [77]:
plt.close('all')

# chech gait segmentation

In [14]:
t[1].gaitSegmentation(check=True)

Outlier peaks (outside 3 std range) are: [  0  18  19 267 474 512]
peakDiffMean = 175.223255814
peakDiffStd = 7.63629147815


In [15]:
ma.shape

NameError: name 'ma' is not defined